In [3]:
# 1. Create a conda environment

# !conda create -y --name bedrock-guardrails python=3.11.8
# !conda init && activate bedrock-guardrails
# !conda install -n bedrock-guardrails ipykernel --update-deps --force-reinstall -y

Channels:
 - defaults
Platform: linux-64
Solving environment: | Channels:
 - defaults
Platform: linux-64
Solving environment: done
done

## Package Plan ##

  environment location: /home/sagemaker-user/.conda/envs/bedrock-guardrails

  added / updated specs:
    - ipykernel





Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
# 2. Install dependencies

!pip install -r requirements.txt

  Using cached boto3-1.34.162-py3-none-any.whl.metadata (6.6 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached botocore-1.34.162-py3-none-any.whl.metadata (5.7 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.20.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3

In [16]:
# 3. Import necessary libraries and load environment variables
import json
from dotenv import load_dotenv, find_dotenv, set_key
import os
import boto3


# loading environment variables that are stored in local file
local_env_filename = 'bedrock-guardrails.env'
load_dotenv(find_dotenv(local_env_filename),override=True)

os.environ['REGION'] = os.getenv('REGION')
os.environ['AWS_BEDROCK_GUARDRAIL_ID'] = os.getenv('AWS_BEDROCK_GUARDRAIL_ID')
REGION = os.environ['REGION']
AWS_BEDROCK_GUARDRAIL_ID = os.environ['AWS_BEDROCK_GUARDRAIL_ID']

bedrock_runtime_client = boto3.client(service_name='bedrock-runtime', region_name=REGION)
bedrock_client = boto3.client(service_name='bedrock', region_name=REGION)

MODEL_ID = "amazon.titan-text-express-v1" #"anthropic.claude-3-haiku-20240307-v1:0" # "anthropic.claude-3-5-sonnet-20240620-v1:0" "meta.llama3-1-70b-instruct-v1:0"

# source: https://medium.com/@vinayakdeshpande111/aws-guardrails-with-langchain-6554411d4d74

In [17]:
# Create Bedrock Guardrail

import json, random, string

response = bedrock_client.create_guardrail(
    name="Guardrail-from-Code-" + "".join(random.choices(string.ascii_lowercase, k=8)),
    description='string',
    topicPolicyConfig={
        'topicsConfig': [
            {
                "name": "Bitcoin",
                "definition": "Providing advice, direction, or examples of how to mine, use, or interact with Bitcoin, including Cryptocurrency-related third-party services.",
                "examples": [
                    "How do I mine Bitcoin?",
                    "What is the current value of BTC?",
                    "Which instance is the best for crypto mining?",
                    "Is mining cryptocurrency against the terms?",
                    "How do I maximize my Bitcoin profits?",
                ],
                "type": "DENY",
            }
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {"type": "SEXUAL", "inputStrength": "HIGH", "outputStrength": "HIGH"},
            {"type": "HATE", "inputStrength": "HIGH", "outputStrength": "HIGH"},
            {"type": "VIOLENCE", "inputStrength": "HIGH", "outputStrength": "HIGH"},
            {"type": "INSULTS", "inputStrength": "HIGH", "outputStrength": "HIGH"},
            {"type": "MISCONDUCT", "inputStrength": "HIGH", "outputStrength": "HIGH"},
            {"type": "PROMPT_ATTACK", "inputStrength": "HIGH", "outputStrength": "NONE"},
        ]
    },
    wordPolicyConfig={
        "wordsConfig": [{"text": "AnyCompany"}],
        "managedWordListsConfig": [{"type": "PROFANITY"}],
    },
    sensitiveInformationPolicyConfig={
        "piiEntitiesConfig": [
            {"type": "NAME", "action": "ANONYMIZE"},
            {"type": "EMAIL", "action": "ANONYMIZE"},
        ],
    },
    blockedInputMessaging="Apologies, this model cannot be used to discuss inappropriate or off-topic content.",
    blockedOutputsMessaging="Apologies, the model's response to your request was blocked.",
)


os.environ['AWS_BEDROCK_GUARDRAIL_ID'] = response['guardrailId']
AWS_BEDROCK_GUARDRAIL_ID = os.environ['AWS_BEDROCK_GUARDRAIL_ID']
print(f'AWS_BEDROCK_GUARDRAIL_ID: {AWS_BEDROCK_GUARDRAIL_ID}')
# Update environment variable
set_key(local_env_filename, 'AWS_BEDROCK_GUARDRAIL_ID', AWS_BEDROCK_GUARDRAIL_ID)

AWS_BEDROCK_GUARDRAIL_ID: 6mjrqx7shadv


(True, 'AWS_BEDROCK_GUARDRAIL_ID', '6mjrqx7shadv')

In [23]:
# Create a guardrail version
response = bedrock_client.create_guardrail_version(
    guardrailIdentifier=AWS_BEDROCK_GUARDRAIL_ID,
)
VERSION = response.get('version')
print(f'VERSION: {VERSION}')

VERSION: 3


In [29]:
# Invoking model with guardrail directly

import json

prompt = "How do I mine Bitcoin?"

body = {
    "inputText": prompt,
    "textGenerationConfig": {
        "maxTokenCount": 512,
        "stopSequences": [],
        "temperature": 0,
        "topP": 0.9
    }
}

response = bedrock_runtime_client.invoke_model(
    body=json.dumps(body),
    modelId=MODEL_ID,
    contentType="application/json",
    accept="application/json",
    guardrailIdentifier=AWS_BEDROCK_GUARDRAIL_ID,
    guardrailVersion=VERSION,
)

response_body = json.loads(response.get('body').read()) # read the response

response_text = response_body['results'][0]['outputText']

print(response_text)

Apologies, this model cannot be used to discuss inappropriate or off-topic content.


In [25]:
# Initialize Langchain LLM wrapper

from langchain_aws import BedrockLLM

llm = BedrockLLM(
    model_id=MODEL_ID,
    model_kwargs={},
    guardrails={"guardrailIdentifier": AWS_BEDROCK_GUARDRAIL_ID, "guardrailVersion": VERSION, "trace": True},
    region_name= REGION
)

In [26]:
# Create Langchain prompt template
from langchain_core.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant." 
                "You answer the questions based on your knowledge." 
                "Return only the answer without any prefix or suffix."),
    ("human", "Question : ```{imput}```")
])

In [27]:
# Create chain
from langchain_core.output_parsers import StrOutputParser

chain = prompt_template | llm | StrOutputParser()

In [30]:
# Test Guardrails
 
result = chain.invoke(input = 'How do I mine Bitcoin?')

print(f'result: {result}')

result = chain.invoke(input = 'My email address is fhuthmacher@gmail.com')
print(f'result: {result}')

result: Apologies, this model cannot be used to discuss inappropriate or off-topic content.
result: 
{NAME}: The email address you provided is invalid.


### Conclusion

Amazon Bedrock Guardrails are a way for you to implement safeguards tailored to your application requirements and aligned with your responsible AI policies. You can use Guardrails with models on Amazon Bedrock or with any 3rd Party or self hosted model. 
Guardrails enable you to configure responsible AI filters, block specific topics in simple natural language, block or mask sensitive information, and help filter and reduce irrelevant or hallucinated response.


### Appendix
1) Bedrock Guardrails workshop: https://catalog.workshops.aws/building-with-amazon-bedrock/en-US/security-safeguards
2) Youtube Video: https://www.youtube.com/watch?v=srQxO_o9KgM
3) Bedrock Guardrails pricing: https://aws.amazon.com/bedrock/pricing/